## Exercise 2 - Automating Excel Reports

In [8]:
import pandas as pd

# Load Excel file into DataFrame 
df = pd.read_excel("monthly-sales.xlsx")

# Inspect data 
print(df.head(20))
df.info()

      Month  Monthly Sales
0   2008-01         154000
1   2008-02          96000
2   2008-03          73000
3   2008-04          51000
4   2008-05          53000
5   2008-06          59000
6   2008-07          95000
7   2008-08         169000
8   2008-09         210000
9   2008-10         278000
10  2008-11         301000
11  2008-12         245000
12  2009-01         200000
13  2009-02         118000
14  2009-03          90000
15  2009-04          84000
16  2009-05          77000
17  2009-06          91000
18  2009-07         167000
19  2009-08         169000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Month          69 non-null     object
 1   Monthly Sales  69 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ KB


In [9]:
# Group the data month and calculate total sales
grouped_sales = df.groupby('Month')['Monthly Sales'].sum().reset_index()

In [11]:
# Export to same excel file in new sheet (Monthly Summary)

with pd.ExcelWriter("monthly-sales.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sales', index=False)
    grouped_sales.to_excel(writer, sheet_name="Monthly Summary", index=False)

In [ ]:
# Format header row to be bold and apply a background color for better readability

from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill

# Open workbook
wb = load_workbook('monthly-sales.xlsx')
ws = wb['Monthly Summary']

# Styles
header_font = Font(bold=True, color ="FFFFFF")
header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

# Iterate through cells of row 1 
for cell in ws[1]:
     cell.font = header_font 
     cell.fill = header_fill 

# Save report and close workbook
wb.save('monthly_sales_report.xlsx')
wb.close()

## Exercise 3 - Advanced Workbook Manipulation

In [16]:
# Dataset 

data = {
    "Sales": [
    ['Product ID', 'Product Name', 'Quantity Sold', 'Total Revenue'],
    [101, 'Widget A', 150, 3000],
    [102, 'Widget B', 200, 5000],
    [103, 'Widget C', 120, 3600],
    [104, "Widget D", 80, 2400]
    ],

    "Inventory": [
    ['Product ID', 'Product Name', 'Stock Quantity', 'Reorder Level'],
    [101, 'Widget A', 300, 50],
    [102, 'Widget B', 150, 30],
    [103, 'Widget C', 200, 40],
    [104, 'Widget D', 100, 20]
    ],

    "Customers": [
    ['Customer ID', 'Customer Name', 'Contact Information'],
    ['001', 'John Smith', 'john.smith@example.com'],
    ['002', 'Jane Doe', 'jane.doe@example.com'],
    ['003', 'Mike Johnson', 'mike.j@example.com'],
    ['004', 'Emily Brown', 'emily.brown@example.com']
]
}

In [23]:
from openpyxl import Workbook 
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Border, Side

wb = Workbook()

# Define border style
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')

)

# Iterate through the data for different sheets 
for index, (sheet_title, rows) in enumerate(data.items()):

    # If 1st sheet, make it active and change title , if not make a new sheet
    if index == 0:
        ws = wb.active
        ws.title = sheet_title
    else:
        ws = wb.create_sheet(sheet_title)

    # Append rows to sheet 
    for row in rows:
        ws.append(row)

    # Auto-adjust column widths 
    for col_cells in ws.columns:
        values = [str(cell.value) if cell.value else "" for cell in col_cells]
        max_length = max(map(len, values))
        ws.column_dimensions[get_column_letter(col_cells[0].column)].width = max_length + 2
    
    # Make headers bold 
    for cell in ws[1]:
        cell.font = Font(bold=True)

    # Get the filled data range for current sheet
    min_row = ws.min_row
    max_row = ws.max_row
    min_col = ws.min_column
    max_col = ws.max_column

    # Apply borders to that range 
    for row in ws.iter_rows(
        min_row=min_row,
        max_row=max_row,
        min_col=min_col,
        max_col=max_col
    ):
        for cell in row:
            cell.border = thin_border  


wb.save('business_data.xlsx')

### Exercise 3 - Refactored Version with Functions (Clenar Look)

In [24]:
# Define a thin border style (can reuse for all sheets)
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Function to add data to a sheet
def add_data_to_sheet(ws, rows):
    """Append rows to the worksheet"""
    for row in rows:
        ws.append(row)

# Function to auto-adjust column widths
def auto_adjust_column_widths(ws):
    """Adjust column widths based on the longest value in each column"""
    for col_cells in ws.columns:
        values = [str(cell.value) if cell.value else "" for cell in col_cells]
        max_length = max(map(len, values))
        ws.column_dimensions[get_column_letter(col_cells[0].column)].width = max_length + 2

# Function to style headers (bold + borders)
def style_headers(ws):
    """Make the first row bold"""
    for cell in ws[1]:
        cell.font = Font(bold=True)


# Function to apply borders to the filled data range
def apply_borders(ws):
    """Apply thin border to all filled cells in the sheet."""
    for row in ws.iter_rows(min_row=ws.min_row, max_row=ws.max_row,
                            min_col=ws.min_column, max_col=ws.max_column):
        for cell in row:
            cell.border = thin_border

# Create workbook
wb = Workbook()

# Iterate through sheets and data
for index, (sheet_title, rows) in enumerate(data.items()):
    if index == 0:
        ws = wb.active
        ws.title = sheet_title
    else:
        ws = wb.create_sheet(sheet_title)

    add_data_to_sheet(ws, rows)
    auto_adjust_column_widths(ws)
    style_headers(ws)
    apply_borders(ws)

# Save workbook
wb.save('business_data.xlsx')


## Exercise 4 - Data Visualization In Excel

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.chart import BarChart, Reference 

# Load workbook and create new sheet 
wb = load_workbook('Excel\ -\ Sales\ Performance\ Dashboard.xlsx')
ws = wb.create_sheet(title='Performance')

# Sales Data sheet 
ws_sales_data = wb['Sales Data']

# Define data range for chart 
data = Reference(ws_sales_data, )




wb.save('')
